<a href="https://colab.research.google.com/github/AngeMervaneJdev/ML_AI_1/blob/main/SimplConv1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#[Problème 1] Création d'une classe de couche convolutionnelle unidimensionnelle qui limite le nombre de canaux à un

In [2]:
import numpy as np;
from scipy.sparse import dia_matrix

In [34]:

class SimpleConv1d:
    def __init__(self, weight, baias):
        self.optimizer = None

        self.W = weight
        self.B = baias
        self.dW = None
        self.dB = None
        self.idx =None
        self.X = None
        pass

    def forward(self, X):
        self.a = X.copy()
        idx1 = np.arange(self.W.shape[0])
        idx2 = np.arange(self.W.shape[0]-1 ).reshape(-1, 1)
        self.idx = idx1 + idx2
        A = np.sum(X[self.idx] * self.W.T,axis=1) + self.B
        return A
    def backward(self, dX,dA):

        dB = np.sum(dA,axis=0)
        dW = np.sum(dA[:,np.newaxis] *dX[self.idx],axis=0)
        da = dA.reshape(-1,1)
        data= np.repeat(da,dX.shape[0],axis=1)
        offsets= np.arange(self.output(data))
        d = dia_matrix((data,offsets),shape=(w.shape[0],x.shape[0])).toarray()
        dx = np.sum(d * w[:,np.newaxis],axis=0)
        
        return dX
    def output(self,X):
        n_in = X.shape[1]
        p = 0
        f = self.W.shape[0]
        s = 1 
        n_out = ((n_in + 2*p - f)/s)+1
        return n_out

#[Problème 2] Calcul de la taille de sortie après convolution unidimensionnelle

In [7]:
def output(X,p,f,s):
    n_in = X.shape[0]
    n_out = ((n_in + 2*p - f)/s)+1
    return n_out

#[Problème 3] Expérience d'une couche convolutionnelle unidimensionnelle avec un petit réseau

In [11]:
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])
x.shape,w.shape,b.shape

((4,), (3,), (1,))

In [9]:
sc1 = SimpleConv1d(w,b)
sc1.forward(x)

array([35, 50])

In [22]:
delta_a = np.array([10, 20])
delta_a.shape
delta_b = np.array([30])
delta_w = np.array([50, 80, 110])
delta_x = np.array([30, 110, 170, 140])

In [35]:
sc1 = SimpleConv1d(delta_w,delta_b)
sc1.backward(delta_x,delta_a)

array([ 30, 110, 170, 140])

In [37]:
x = np.array([1, 2, 3, 4])
w = np.array([3, 5, 7])

a = np.empty((2, 3))

indexes0 = np.array([0, 1, 2])
indexes1 = np.array([1, 2, 3])

a[0] = x[indexes0]*w # x[indexes0]は([1, 2, 3])である
a[1] = x[indexes1]*w # x[indexes1]は([2, 3, 4])である

a = a.sum(axis=1)
a

array([34., 49.])

#[Problème 4] Création d'une classe de couche convolutionnelle unidimensionnelle qui ne limite pas le nombre de canaux

In [49]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3])


In [50]:
def output2(X,p,f,s):
    n_in = X.shape[1]
    p = p #バディング
    f = f #フィルタのサイズ
    s = s #ストライドのサイズ
    n_out = ((n_in + 2*p - f)/s)+1
    return n_out

In [51]:

a = np.array([[16, 22], [17, 23], [18, 24]])


o = int(output2(x,0,w.shape[-1],1))
print(o)

nin, i = x.shape
print(nin,i)
#oc出力チャンネル , cチャンネル数 , fフィルタサイズ
oc, c, f = w.shape
print(oc,c,f)

#c * f 
idx_arr = np.arange(c*f).reshape(c,f)
idx_arr

2
2 4
3 2 3


array([[0, 1, 2],
       [3, 4, 5]])

In [52]:
#フォワード
idx1 = np.arange(w.shape[-1])

idx2 = np.arange(w.shape[-1]-1 ).reshape(-1, 1)

idx = idx1 + idx2

print(x[:,idx].shape)
a1 = x[:,idx]
a2 = w
print(a1)
a= np.zeros((f,o))
for i in range(f):
    for j in range(o):
        x_in = a1[j]
        print(x_in.shape)
        a[i][j] = np.sum(x_in @ w[i][j])
        print('a',a.shape)
    print(b[i].shape)
    a[i] += a[i] + b[i]
    print(a)

(2, 2, 3)
[[[1 2 3]
  [2 3 4]]

 [[2 3 4]
  [3 4 5]]]
(2, 3)
a (3, 2)
(2, 3)
a (3, 2)
()
[[31. 43.]
 [ 0.  0.]
 [ 0.  0.]]
(2, 3)
a (3, 2)
(2, 3)
a (3, 2)
()
[[31. 43.]
 [32. 44.]
 [ 0.  0.]]
(2, 3)
a (3, 2)
(2, 3)
a (3, 2)
()
[[31. 43.]
 [32. 44.]
 [33. 45.]]


In [ ]:
class Conv1d:
    def __init__(self,ch, f_num, f_size, initializer,optimizer):
        self.optimizer = optimizer

        self.W = initializer.W(f_num=f_num,f_size=f_size,ch=ch)
        self.B = initializer.B(f_num)
        self.dW = None
        self.dB = None
        self.idx =None
        self.X = None
        self.A = None
        pass

    def output(self,X,p,f,s):
        self.x_in = X.shape[-1]
        n_out = int(((self.x_in + 2*p - f)/s)+1)
        return n_out

    def forward(self, X):
        #print(X.shape)
        self.X = X
        #shape
        i_ch,featu = self.X.shape
        o_ch,i_ch,f_size = self.W.shape
        self.o_size = self.output(X,0,f_size,1)
        A = np.zeros((o_ch,self.o_size))

        idx1 = np.arange(f_size)
        idx2 = np.arange(f_size-1 ).reshape(-1, 1)
        idx = idx1 + idx2
        self.xs = X[:,idx]
        for oc in range(o_ch):
            for ic in range(i_ch):
                for os in range(self.o_size):
                    A[oc , os] += np.sum(self.X[ic, os:os+f_size] * self.W[oc, ic, :])
        A += self.B.reshape(-1,1)
        #print(A.shape)
        return A

    def backward(self, dA):
        i_ch,featu = self.X.shape
        o_ch,i_ch,f_size = self.W.shape
        #bias
        self.dB = np.sum(dA,axis=1)

        self.dW = np.zeros(self.W.shape)
        for oc in range(o_ch):
            for ic in range(i_ch):
                for f in range(f_size):
                    for os in range(self.o_size):
                        self.dW[oc,ic,f] += dA[ic,os] * self.X[ic,f + os]
        #print('dB',self.dB.shape)
        #print('dW',self.dW.shape)
        #backpropa
        dX = 0
        for i in range(o_ch):
            da = dA[:,:,np.newaxis]
            data= np.repeat(da[i],i_ch,axis=1)
            offsets= np.arange(self.o_size)
            #print(offsets)
            da = dia_matrix((data,offsets),shape=(f_size,i_ch)).toarray()
            dX += np.sum(da * self.W[i][:,:,np.newaxis],axis=1)

         # 更新
        self = self.optimizer.update(self)
        return dX